In [10]:
from sklearn.datasets import fetch_openml
import torch

mnist = fetch_openml('mnist_784', as_frame=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = torch.from_numpy(mnist.data).float().to(device)
Y = torch.from_numpy(mnist.target.astype(int)).to(device)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

split = int(len(X) * 6 / 7)

train_ds = TensorDataset(X[:split].view(-1, 1, 28, 28), Y[:split])
test_ds = TensorDataset(X[split:].view(-1, 1, 28, 28), Y[split:])
train_loader = DataLoader(train_ds, batch_size=6250)
test_loader = DataLoader(test_ds, batch_size=1250)

In [18]:
import torch.nn.functional as F
import torch.nn as nn

class MulticlassCCN(nn.Module):
    def __init__(self, c1=32, c2=64, fc_hidden=128, device=device):
        """
        c1: number of channels after first conv layer
        c2: number of channels after second conv layer
        fc_hidden: number of hidden units in the fully connected layer
        """
        super().__init__()
        self.conv1 = nn.Conv2d(1, c1, kernel_size=3, padding=1, device=device)
        self.conv2 = nn.Conv2d(c1, c2, kernel_size=3, padding=1, device=device)
        self.fc1 = nn.Linear(c2 * 7 * 7, fc_hidden, device=device)
        self.fc2 = nn.Linear(fc_hidden, 10, device=device)

    def forward(self, X):
        X = F.relu(self.conv1(X))       # (N, c1, 28, 28)
        X = F.max_pool2d(X, 2, 2)       # (N, c1, 14, 14)
        X = F.relu(self.conv2(X))       # (N, c2, 14, 14)
        X = F.max_pool2d(X, 2, 2)       # (N, c2, 7, 7)
        X = X.view(X.size(0), -1)       # flatten
        X = F.relu(self.fc1(X))         # (N, fc_hidden)
        X = self.fc2(X)                 # (N, 10)
        return X


In [ ]:
model = MulticlassCCN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100

for i in range(epochs):
    for train_X, train_Y in train_loader:
        loss = criterion(model(train_X), train_Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


KeyboardInterrupt: 

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        outputs = model(X_batch)              # logits, shape (batch, 10)
        preds = torch.argmax(outputs, dim=1)  # predicted classes

        y_true.extend(y_batch.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

acc  = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec  = recall_score(y_true, y_pred, average='macro')
f1   = f1_score(y_true, y_pred, average='macro')

print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

Accuracy:  0.9863
Precision: 0.9862
Recall:    0.9863
F1-score:  0.9862
